In [19]:
import math
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf

In [20]:
secondary_difference = 0.39

In [21]:
primary_2000_sqm = int(8700/29.7)
secondary_2000_sqm = int(primary_2000_sqm*secondary_difference)
primary_2020_sqm = int(86100/88)
secondary_2020_sqm = int(primary_2020_sqm*secondary_difference)

Методика подсчета: https://mortgage-calculator.ru/формула-расчета-ипотеки/

In [22]:
def calculate_annuity_mortgage(interest_rate_yearly, loan_term_yearly, loan_amount, rest_debt):
    monthly_rate = interest_rate_yearly / 12/100
    loan_term_montly = loan_term_yearly*12
    first_payment = loan_amount- rest_debt
    
    rd = rest_debt
    me = []
    mp = []
    
    for i in range(loan_term_montly):
        main_rate = (1 + monthly_rate)**(loan_term_montly-i)
        monthly_payment = rd*monthly_rate*main_rate/(main_rate-1)
        # print(monthly_rate)
        interest_to_pay = rd * monthly_rate
        monthly_main_loan_to_pay = monthly_payment - interest_to_pay
        monthly_extra = (monthly_payment)*(loan_term_montly-i) - rd
        me.append(interest_to_pay)
        mp.append(monthly_payment)
        rd -= monthly_main_loan_to_pay
    
    total = int(sum(me))
    mo_p = int(sum(mp))
    extra = "или " + str(int(total*100/loan_amount)) + "%"
    b = str(total)
    op = str(int(mo_p+first_payment))
    ta = "Общая выплаченная сумма " + op[:-6] + " " + op[-6:-3] + " " + op[-3:] + " руб"
    c = " Переплата " + b[:-6] + " " + b[-6:-3] + " " + b[-3:] + " руб. "
    return ta + c + extra
    

In [23]:
property_value = 10000000
initial_deposit_percentage = 0.15
interest_rate = 5
loan_term = 20
loan_amount = property_value - property_value*initial_deposit_percentage

In [24]:
property_value*initial_deposit_percentage

1500000.0

In [25]:
calculate_annuity_mortgage(interest_rate_yearly=interest_rate, loan_term_yearly=loan_term, loan_amount=property_value, rest_debt=loan_amount)

'Общая выплаченная сумма 14 963 097 руб Переплата 4 963 097 руб. или 49%'

https://www.irn.ru/calc/?todo=calc&service_key=64b26d6a56d2c3.90525403&calc_type=payment&pay_type=0&full_price=10+000+000&month_payment=60+000&price_currency=0&first_payment=1+500+000&first_payment_currency=0&first_payment_pr=15%2C0&wage=5%2C0&wage_currency=0&years=20&submit=%D0%E0%F1%F1%F7%E8%F2%E0%F2%FC#results

In [26]:
def calculate_compounded_amount(amount_invested, r, m, t,):
    """
    amount_invested = 100 for $100
    r = interest_rate 0.05 for 5%
    m = compounding_frequency 1
    t = time 2 as T
    """
    r_m = m * (math.exp(r/m) - 1)
    compounded_amount = amount_invested * (1 + r_m/m)**(m*t)
    profit = ", рост на " + str(int(compounded_amount *100 / amount_invested - 100)) +"%"
    ca = str(int(compounded_amount))
    c = ca[:-6] + " " + ca[-6:-3] + " "+ ca[-3:]
    return c + profit 

In [27]:
amount_invested = 1500000
r = 0.054
m = 1
t = 20

In [28]:
compounded_amount = calculate_compounded_amount(amount_invested=amount_invested, r=r, m=m, t=t)
print("Депозит под 5% на 20 лет без доп вложений: " + compounded_amount)

Депозит под 5% на 20 лет без доп вложений: 4 417 019, рост на 194%


In [50]:
def format_total_amount(amount):
    return '{:,.2f}'.format(amount).replace(',', ' ')

In [54]:
def calculate_compound_interest(invested_amount, yearly_return, duration, additional_monthly, additional_yearly):
    yearly_return = yearly_return / 100
    compound_duration = duration * 12

    total_amount = invested_amount
    
    end_date = datetime.today().date()
    start_date = end_date - pd.DateOffset(years=duration) - timedelta(days=1)
    usdrub_data = yf.download('RUB=X', start=start_date, end=end_date, progress=False)
    exchange_rate_df = usdrub_data[['Close']]
    exchange_rate_df.columns = ['exchange_rate']

    monthly_exchange_rate_df = exchange_rate_df.resample('M').first().reset_index()

    monthly_exchange_rate_df = monthly_exchange_rate_df.dropna(subset=['exchange_rate'])

    total_amount_usd = total_amount / monthly_exchange_rate_df.iloc[0]['exchange_rate']

    monthly_total_amount = [total_amount_usd]
    mtarub = [invested_amount]
    
    for month in range(len(monthly_exchange_rate_df) - 1):
        rate = monthly_exchange_rate_df.iloc[month]['exchange_rate']
        monthly_return_usd = total_amount_usd * yearly_return / 12
        monthly_deposit_usd = (additional_monthly + additional_yearly / 12) / rate
        total_amount_usd += monthly_return_usd + monthly_deposit_usd
        monthly_total_amount.append(total_amount_usd)
        mtarub.append(total_amount_usd*rate)

    mta = {
        'Date': monthly_exchange_rate_df['Date'],
        'Total Amount (RUB)': [format_total_amount(amount) for amount in mtarub],
        'Total Amount (USD)': [format_total_amount(amount) for amount in monthly_total_amount],
        'Exchange rate': monthly_exchange_rate_df['exchange_rate']
    }

    df = pd.DataFrame(mta)

    return df


In [52]:
#case1
invested_amount = 1500000
yearly_return = 8
duration = 20
additional_monthly = 56000
additional_yearly = 0

In [56]:
#case2
invested_amount = 1500000
yearly_return = 8
duration = 20
additional_monthly = 15000
additional_yearly = 40000

In [57]:
m = calculate_compound_interest(invested_amount, yearly_return, duration, additional_monthly, additional_yearly)
m

,Date,Total Amount (RUB),Total Amount (USD),Exchange rate
0,2003-12-31,1 500 000.00,50 505.05,29.700001
1,2004-01-31,1 528 333.33,51 459.03,29.230000
2,2004-02-29,1 532 508.51,52 429.30,28.590000
3,2004-03-31,1 527 280.14,53 420.08,28.520000
4,2004-04-30,1 552 031.03,54 419.04,28.487801
...,...,...,...,...
231,2023-03-31,36 667 128.04,519 732.48,75.014999
232,2023-04-30,39 265 983.21,523 441.76,77.964996
233,2023-05-31,41 100 535.70,527 166.52,80.415001
234,2023-06-30,42 693 043.39,530 908.95,81.514999


In [34]:
m.iloc[92]

Date                  2011-11-30 00:00:00
Total Amount (RUB)         5667373.693124
Total Amount (USD)          176010.311037
Exchange rate                   30.304001
Name: 95, dtype: object

In [35]:
4500/1500

3.0

In [36]:
90/29

3.103448275862069

In [37]:
115000/5000

23.0